In [1]:
!nvidia-smi

Tue Jul  2 16:13:28 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   41C    P8              8W /  280W |     297MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score,ConfusionMatrixDisplay,confusion_matrix,precision_score,recall_score,roc_curve,roc_auc_score,balanced_accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import datetime
from dataclasses import dataclass
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from scipy.ndimage import gaussian_filter1d
from itertools import combinations as comb
from pyts.image import RecurrencePlot
import PIL
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,StratifiedKFold
from unidecode import unidecode

In [3]:
train_data_embedding = pd.read_csv('X_train_electra_embedding.csv')
y_train = pd.read_csv('y_train.csv')

test_data_embedding = pd.read_csv('X_test_electra_embedding.csv')
y_test = pd.read_csv('y_test.csv')

In [4]:
X_train = train_data_embedding
y_train = y_train

X_test = test_data_embedding
y_test = y_test

In [5]:
from sklearn.linear_model import LogisticRegression
Logit = LogisticRegression(C=1e2,
                          multi_class='ovr',
                          random_state=17,
                          max_iter=200
                          )
Logit.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(C=100.0, max_iter=200, multi_class='ovr', random_state=17)

In [6]:
y_test_pred = Logit.predict(X_test)
y_test_pred_proba = Logit.predict_proba(X_test)
print(y_test_pred[:5])
print(y_test_pred_proba[:5])

[1 0 0 1 1]
[[0.33530887 0.66469113]
 [0.75314365 0.24685635]
 [0.81171355 0.18828645]
 [0.24007107 0.75992893]
 [0.22157966 0.77842034]]


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5786858089453341
[[623 374]
 [389 425]]
              precision    recall  f1-score   support

           0     0.6156    0.6249    0.6202       997
           1     0.5319    0.5221    0.5270       814

    accuracy                         0.5787      1811
   macro avg     0.5738    0.5735    0.5736      1811
weighted avg     0.5780    0.5787    0.5783      1811



In [8]:
### 3. MLP Classifier

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=17, max_iter=1000).fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
y_test_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.484815019326339
[[150 847]
 [ 86 728]]
              precision    recall  f1-score   support

           0     0.6356    0.1505    0.2433       997
           1     0.4622    0.8943    0.6095       814

    accuracy                         0.4848      1811
   macro avg     0.5489    0.5224    0.4264      1811
weighted avg     0.5577    0.4848    0.4079      1811



In [10]:
### 4. KNN

from sklearn.neighbors import KNeighborsClassifier

# 5개 근접한 이웃, 거리측정기준 : 유클리드
knn = KNeighborsClassifier(n_neighbors=5, p=2)

# Model Fitting 과정
knn.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [11]:
y_test_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5444505797901712
[[572 425]
 [400 414]]
              precision    recall  f1-score   support

           0     0.5885    0.5737    0.5810       997
           1     0.4934    0.5086    0.5009       814

    accuracy                         0.5445      1811
   macro avg     0.5410    0.5412    0.5410      1811
weighted avg     0.5458    0.5445    0.5450      1811



In [12]:
### 5. Random Forest

from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=5, random_state=17)
rf_model.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(n_estimators=5, random_state=17)

In [13]:
y_test_pred = rf_model.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5140806184428492
[[533 464]
 [416 398]]
              precision    recall  f1-score   support

           0     0.5616    0.5346    0.5478       997
           1     0.4617    0.4889    0.4749       814

    accuracy                         0.5141      1811
   macro avg     0.5117    0.5118    0.5114      1811
weighted avg     0.5167    0.5141    0.5150      1811



In [14]:
### 6. Decision Tree
from sklearn.tree import DecisionTreeClassifier
DT_MODEL= DecisionTreeClassifier(random_state=17)
DT_MODEL.fit(X_train, y_train)

DecisionTreeClassifier(random_state=17)

In [15]:
y_test_pred = DT_MODEL.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5096631695196024
[[516 481]
 [407 407]]
              precision    recall  f1-score   support

           0     0.5590    0.5176    0.5375       997
           1     0.4583    0.5000    0.4783       814

    accuracy                         0.5097      1811
   macro avg     0.5087    0.5088    0.5079      1811
weighted avg     0.5138    0.5097    0.5109      1811



In [16]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_xgb = le.fit_transform(y_train)
le = LabelEncoder()
y_test_xgb = le.fit_transform(y_test)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
xgb_class = XGBClassifier()
xgb_class.fit(X_train, y_train_xgb)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [19]:
y_test_pred = xgb_class.predict(X_test)
print(accuracy_score(y_test_xgb, y_test_pred))
print(confusion_matrix(y_test_xgb, y_test_pred))
print(classification_report(y_test_xgb, y_test_pred, digits=4))

0.5632247377139702
[[598 399]
 [392 422]]
              precision    recall  f1-score   support

           0     0.6040    0.5998    0.6019       997
           1     0.5140    0.5184    0.5162       814

    accuracy                         0.5632      1811
   macro avg     0.5590    0.5591    0.5591      1811
weighted avg     0.5636    0.5632    0.5634      1811



In [20]:
### 8. LightGBM
from lightgbm import LGBMClassifier
lgbm_class = LGBMClassifier()
lgbm_class.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier()

In [21]:
y_test_pred =lgbm_class.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5670900055218111
[[603 394]
 [390 424]]
              precision    recall  f1-score   support

           0     0.6073    0.6048    0.6060       997
           1     0.5183    0.5209    0.5196       814

    accuracy                         0.5671      1811
   macro avg     0.5628    0.5628    0.5628      1811
weighted avg     0.5673    0.5671    0.5672      1811



In [22]:
from catboost import CatBoostClassifier
cat_class = CatBoostClassifier()
cat_class.fit(X_train, y_train)

Learning rate set to 0.014738
0:	learn: 0.6921241	total: 348ms	remaining: 5m 47s
1:	learn: 0.6908852	total: 485ms	remaining: 4m 2s
2:	learn: 0.6900567	total: 583ms	remaining: 3m 13s
3:	learn: 0.6891691	total: 725ms	remaining: 3m
4:	learn: 0.6880485	total: 875ms	remaining: 2m 54s
5:	learn: 0.6872049	total: 1.04s	remaining: 2m 52s
6:	learn: 0.6861765	total: 1.18s	remaining: 2m 46s
7:	learn: 0.6852845	total: 1.3s	remaining: 2m 41s
8:	learn: 0.6842578	total: 1.39s	remaining: 2m 33s
9:	learn: 0.6834163	total: 1.51s	remaining: 2m 29s
10:	learn: 0.6824594	total: 1.63s	remaining: 2m 26s
11:	learn: 0.6815206	total: 1.71s	remaining: 2m 20s
12:	learn: 0.6805540	total: 1.89s	remaining: 2m 23s
13:	learn: 0.6798108	total: 1.98s	remaining: 2m 19s
14:	learn: 0.6788781	total: 2.07s	remaining: 2m 16s
15:	learn: 0.6779626	total: 2.16s	remaining: 2m 12s
16:	learn: 0.6770081	total: 2.26s	remaining: 2m 10s
17:	learn: 0.6761706	total: 2.36s	remaining: 2m 8s
18:	learn: 0.6751856	total: 2.46s	remaining: 2m 6s


In [23]:
y_test_pred = cat_class.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5786858089453341
[[619 378]
 [385 429]]
              precision    recall  f1-score   support

           0     0.6165    0.6209    0.6187       997
           1     0.5316    0.5270    0.5293       814

    accuracy                         0.5787      1811
   macro avg     0.5741    0.5739    0.5740      1811
weighted avg     0.5784    0.5787    0.5785      1811



In [24]:
### 10. LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_class = LinearDiscriminantAnalysis()
lda_class.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearDiscriminantAnalysis()

In [25]:
y_test_pred = lda_class.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5433462175593595
[[553 444]
 [383 431]]
              precision    recall  f1-score   support

           0     0.5908    0.5547    0.5722       997
           1     0.4926    0.5295    0.5104       814

    accuracy                         0.5433      1811
   macro avg     0.5417    0.5421    0.5413      1811
weighted avg     0.5467    0.5433    0.5444      1811



In [26]:
### 11. QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
qda_class = QuadraticDiscriminantAnalysis()
qda_class.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

In [27]:
y_test_pred = qda_class.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5659856432909994
[[910  87]
 [699 115]]
              precision    recall  f1-score   support

           0     0.5656    0.9127    0.6984       997
           1     0.5693    0.1413    0.2264       814

    accuracy                         0.5660      1811
   macro avg     0.5674    0.5270    0.4624      1811
weighted avg     0.5672    0.5660    0.4862      1811

